In [78]:
import pandas as pd

In [79]:
df=pd.read_csv("spotify_millsongdata.csv")

In [80]:
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [81]:
df.shape

(57650, 4)

In [82]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [83]:
df=df.sample(10000).drop('link',axis=1).reset_index(drop=True)

In [84]:
df['text'][0]

"Now there's nothing left to do  \r\nBut to lay my head down for sleeping  \r\nI'm weak with exhaustion and the lover's blues  \r\nAnd I still can't forget those weeks I had with you  \r\n  \r\nBut I guess it was only a summer love  \r\nNothing too serious and nothing to last  \r\nI feel like a fool to be longing for that lyin' summer love  \r\n  \r\nOnce we both loved just a bit too hard  \r\nFor deep down we both knew it was dyin'  \r\nAnd we both talked just a little about forever  \r\nBut we both knew we were doing some lyin'.  \r\n  \r\nNow the snow falls, freezing my hurt for you  \r\nBut some nights I feel it's still there.  \r\nI've come back now to the life I've known before you  \r\nBut I doesn't have the same old meaning.\r\n\r\n"

In [85]:
df.shape

(10000, 3)

In [86]:
df['text']=df['text'].str.lower().replace(r'^\w\s', '',regex=True).replace('r\n','',regex=True)

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize

In [88]:
stemmer=PorterStemmer()

In [ ]:
def token(txt):
    token=nltk.word_tokenize(txt)
    a=[stemmer.stem(w)for w in token]
    return " ".join(a) 

In [ ]:
token("you are beautiful,beauty")                         

'you are beauti , beauti'

In [91]:
df['text'].apply(lambda x: token(x))

0       now there 's noth left to do but to lay my hea...
1       in ecstasi they cri to know if their parent lo...
2       if i knew what the nightingal sing in her song...
3       thi is gon na sound a littl obsess a littl bit...
4       just gon na stand there and watch me burn but ...
                              ...                        
9995    lone i wander through scene of my childhood th...
9996    shame shame shame on you miss roxi shame shame...
9997    met her in a club down in old soho where you d...
9998    just hear those sleigh bell jingl ring ting ti...
9999    sittin ' around , wait for an airplan do n't k...
Name: text, Length: 10000, dtype: object

In [92]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
tfidf= TfidfVectorizer(analyzer='word',stop_words='english')


In [ ]:
matrix=tfidf.fit_transform(df['text'])                             

In [ ]:
matrix                                         

<10000x33776 sparse matrix of type '<class 'numpy.float64'>'
	with 534033 stored elements in Compressed Sparse Row format>

In [96]:
similarity=cosine_similarity(matrix)

In [ ]:
similarity[0]                                             

array([1.        , 0.02292606, 0.06111985, ..., 0.00479083, 0.02997934,
       0.02158721])

In [98]:
df[df['song']=="Waiting For The Man"].index[0] #Gives the song ID

8610

In [99]:
df.tail(5)

,artist,song,text
9995,John McDermott,The Old House,lonely i wander through scenes of my childhood...
9996,Aerosmith,"Shame, Shame, Shame",shame shame shame on you miss roxy \r\nshame ...
9997,Kinks,Lola,met her in a club down in old soho \r\nwhere ...
9998,Air Supply,Sleigh Ride,just hear those sleigh bells jingling ring tin...
9999,Widespread Panic,Airplane,"sittin' around, waiting for an airplane \r\nd..."


Recommender Function

In [105]:
def recommender(song_name):
    idx= df[df['song']==song_name].index[0]
    distance= sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x:x[1])
    song= []
    for s_id in distance[1:5]:
        song.append(df.iloc[s_id[0]].song)
    return song

In [106]:
recommender("Waiting For The Man")

['Fat Man', 'Billy', 'In Your Corner', 'Love Is Just Around The Corner']

In [107]:
import pickle

In [110]:
pickle.dump(similarity,open('similarity.pkl','wb'))

In [111]:
pickle.dump(df,open('df.pkl','wb'))